In [1]:
import requests
from bs4 import BeautifulSoup as soup
import lxml

from splinter import Browser
import pandas as pd
import matplotlib.pyplot as plt

import html5lib
import splinter
import time
import datetime as dt
import os

from config import api_key
from sqlalchemy import create_engine

In [2]:
#df=pd.read_html("http://www.planecrashinfo.com/2010/2010",header=0)
#df=df[0]

data_file=("Airplane_Crashes_and_Fatalities_Since_1908.csv")

hist_df=pd.read_csv(data_file)

In [3]:
historical_df=hist_df[["Date","Location","Operator","Aboard","Fatalities","Ground","Type","Summary"]].copy()

In [4]:
historical_df['Date'] =  pd.to_datetime(historical_df['Date'])

In [5]:
historical_df["Aboard"]= pd.to_numeric(historical_df['Aboard'],errors='coerce')
historical_df["Fatalities"]= pd.to_numeric(historical_df['Fatalities'],errors='coerce')

In [6]:
#COLUMN_NAMES=['date','time','location','operator','Flight#','Route','AC type','Registration','cn/ln','Aboard','Fatalities','Ground','Summary']
#df1=pd.DataFrame(columns=COLUMN_NAMES)
years=['2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']
no_of_columns=13
combined_list=[]
for yr in years:
    yrly_url="http://www.planecrashinfo.com/"+yr+"/"+yr
    df=pd.read_html(yrly_url,header=0,encoding='utf-8')
    df=df[0]
    total_rows=len(df)
    for i in range(int(total_rows)):
        url = yrly_url+"-" + str(i+1) +".htm"
        df2=pd.read_html(url,header=0)
        df1=df2[0]
        dict4={}
        df1=df1.rename( columns={"-": "Parameters"})
        for col in range(no_of_columns):
            dict4[df1.iloc[col,0]]=df1.iloc[col,1]
        combined_list.append(dict4)

In [7]:
Final_df=pd.DataFrame(combined_list)
#Final_df
headers = Final_df.dtypes.index
print(headers)

Index(['AC  Type:', 'Aboard:', 'Date:', 'Fatalities:', 'Flight #:', 'Ground:',
       'Location:', 'Operator:', 'Registration:', 'Route:', 'Summary:',
       'Time:', 'cn / ln:'],
      dtype='object')


In [8]:

crash_df=Final_df[["Date:","Location:","Operator:","Aboard:","Fatalities:","Ground:","AC  Type:","Summary:"]].copy()
crash_df=crash_df.rename(columns={"Date:":"Date", "Location:":"Location","Operator:":"Operator","Aboard:":"Aboard","Fatalities:":"Fatalities",\
                                  "Ground:":"Ground","AC  Type:":"Type","Summary:":"Summary"})

In [9]:
crash_df

,Date,Location,Operator,Aboard,Fatalities,Ground,Type,Summary
0,"January 01, 2009","Lubock, Texas",Empire Airlines leased from FedEx,2 (passengers:0 crew:2),2 (passengers:0 crew:2),0,ATR-42-320,Crashed while attempting to land. The probable...
1,"January 04, 2009",Near Houma Louisiana,Petroleum Helicopters Inc,9 (passengers:7 crew:2),8 (passengers:6 crew:2),0,Sikorsky S-76C++,A helicopter bound for offshore oil fields wen...
2,"January 15, 2009","New York, New York",US Airways,155 (passengers:150 crew:5),0 (passengers:0 crew:0),0,Airbus A320-214,The plane was taking off from La Guardia Airpo...
3,"January 15, 2009","Adraskan district, Afghanistan",Afghan Army,13 (passengers:? crew:?),13 (passengers:? crew:?),0,Mil Mi-17,Either shot down or crashed due to bad weather.
4,"February 07, 2009","Manacapuru, Brazil",Aerotaxi Manaus,28 (passengers:26 crew:2),24 (passengers:22 crew:2),0,Bandeirante EMB-110P1,The plane was being used as an air taxi to fer...
5,"February 07, 2009","Trigoria, Italy",Air One Executive,2 (passengers:0 crew:2),2 (passengers:0 crew:2),0,Cessna 650 Citation III,"The plane, heading to Bologna to pick up a med..."
6,"February 12, 2009","Clarence Center, New York",Continental Connection/Colgan Air,49 (passengers:44 crew:5),49 (passengers:44 crew:5),1,Bombardier DHC-8-402 Q400,The commuter plane crashed while attemptiong t...
7,"February 15, 2009","Chanco, Chile",Flight Service,13 (passengers:12 crew:1),13 (passengers:12 crew:1),0,Bell UH-1H,The helicopter was carrying firemen to a nearb...
8,"February 15, 2009","Near Isfahan/Shahin Shahr-Hesa Air Base, Iran",HESA,5 (passengers:0 crew:5),5 (passengers:0 crew:5),0,HESA IrAn-140-100,The aircraft crashed druing a training flight.
9,"February 20, 2009","Luxor, Egypt",Aerolift,5 (passengers:0 crew:5),5 (passengers:0 crew:5),0,Antonov 12V,"While attemping to take off from Luxor, the ca..."


In [10]:
from sqlalchemy import create_engine

In [11]:
## Format the date to be the YYYY-mm-dd format

def getMonth(month):
    Months = ['january','february','march','april','may','june','july','august','september','october','november','december']
    month = month.lower()
    for i,value in enumerate(Months):
        if value == month:
            return i+1
    return 0 


def convert_date(crash_date):
    data = str(crash_date).replace(',','').split(' ')
    date = dt.datetime(int(data[2]),getMonth(data[0]),int(data[1]))
    return date

crash_df['Date'] = crash_df.apply(lambda row: convert_date(row['Date']), axis=1)

crash_df.head()

,Date,Location,Operator,Aboard,Fatalities,Ground,Type,Summary
0,2009-01-01,"Lubock, Texas",Empire Airlines leased from FedEx,2 (passengers:0 crew:2),2 (passengers:0 crew:2),0,ATR-42-320,Crashed while attempting to land. The probable...
1,2009-01-04,Near Houma Louisiana,Petroleum Helicopters Inc,9 (passengers:7 crew:2),8 (passengers:6 crew:2),0,Sikorsky S-76C++,A helicopter bound for offshore oil fields wen...
2,2009-01-15,"New York, New York",US Airways,155 (passengers:150 crew:5),0 (passengers:0 crew:0),0,Airbus A320-214,The plane was taking off from La Guardia Airpo...
3,2009-01-15,"Adraskan district, Afghanistan",Afghan Army,13 (passengers:? crew:?),13 (passengers:? crew:?),0,Mil Mi-17,Either shot down or crashed due to bad weather.
4,2009-02-07,"Manacapuru, Brazil",Aerotaxi Manaus,28 (passengers:26 crew:2),24 (passengers:22 crew:2),0,Bandeirante EMB-110P1,The plane was being used as an air taxi to fer...


In [12]:
def get_member_cnt(total_aboard):
    if total_aboard != '?' :
               print(total_aboard.split())
               s = ' '.join(total_aboard.split())
               print(s)
               aboard_ = s.replace('(','').replace(')','').split(' ')

               if aboard_[0] != '?':
                   aboard_total = aboard_[0]
               else:
                   aboard_total = 'NULL'
    else:
        aboard_total='NULL'
    
    return aboard_total

In [13]:
crash_df

,Date,Location,Operator,Aboard,Fatalities,Ground,Type,Summary
0,2009-01-01,"Lubock, Texas",Empire Airlines leased from FedEx,2 (passengers:0 crew:2),2 (passengers:0 crew:2),0,ATR-42-320,Crashed while attempting to land. The probable...
1,2009-01-04,Near Houma Louisiana,Petroleum Helicopters Inc,9 (passengers:7 crew:2),8 (passengers:6 crew:2),0,Sikorsky S-76C++,A helicopter bound for offshore oil fields wen...
2,2009-01-15,"New York, New York",US Airways,155 (passengers:150 crew:5),0 (passengers:0 crew:0),0,Airbus A320-214,The plane was taking off from La Guardia Airpo...
3,2009-01-15,"Adraskan district, Afghanistan",Afghan Army,13 (passengers:? crew:?),13 (passengers:? crew:?),0,Mil Mi-17,Either shot down or crashed due to bad weather.
4,2009-02-07,"Manacapuru, Brazil",Aerotaxi Manaus,28 (passengers:26 crew:2),24 (passengers:22 crew:2),0,Bandeirante EMB-110P1,The plane was being used as an air taxi to fer...
5,2009-02-07,"Trigoria, Italy",Air One Executive,2 (passengers:0 crew:2),2 (passengers:0 crew:2),0,Cessna 650 Citation III,"The plane, heading to Bologna to pick up a med..."
6,2009-02-12,"Clarence Center, New York",Continental Connection/Colgan Air,49 (passengers:44 crew:5),49 (passengers:44 crew:5),1,Bombardier DHC-8-402 Q400,The commuter plane crashed while attemptiong t...
7,2009-02-15,"Chanco, Chile",Flight Service,13 (passengers:12 crew:1),13 (passengers:12 crew:1),0,Bell UH-1H,The helicopter was carrying firemen to a nearb...
8,2009-02-15,"Near Isfahan/Shahin Shahr-Hesa Air Base, Iran",HESA,5 (passengers:0 crew:5),5 (passengers:0 crew:5),0,HESA IrAn-140-100,The aircraft crashed druing a training flight.
9,2009-02-20,"Luxor, Egypt",Aerolift,5 (passengers:0 crew:5),5 (passengers:0 crew:5),0,Antonov 12V,"While attemping to take off from Luxor, the ca..."


In [14]:
crash_df['Aboard'] = crash_df.apply(lambda row: get_member_cnt(row['Aboard']), axis=1)

['2', '(passengers:0', 'crew:2)']
2 (passengers:0 crew:2)
['9', '(passengers:7', 'crew:2)']
9 (passengers:7 crew:2)
['155', '(passengers:150', 'crew:5)']
155 (passengers:150 crew:5)
['13', '(passengers:?', 'crew:?)']
13 (passengers:? crew:?)
['28', '(passengers:26', 'crew:2)']
28 (passengers:26 crew:2)
['2', '(passengers:0', 'crew:2)']
2 (passengers:0 crew:2)
['49', '(passengers:44', 'crew:5)']
49 (passengers:44 crew:5)
['13', '(passengers:12', 'crew:1)']
13 (passengers:12 crew:1)
['5', '(passengers:0', 'crew:5)']
5 (passengers:0 crew:5)
['5', '(passengers:0', 'crew:5)']
5 (passengers:0 crew:5)
['134', '(passengers:127', 'crew:7)']
134 (passengers:127 crew:7)
['11', '(passengers:7', 'crew:4)']
11 (passengers:7 crew:4)
['18', '(passengers:16', 'crew:2)']
18 (passengers:16 crew:2)
['14', '(passengers:13', 'crew:1)']
14 (passengers:13 crew:1)
['2', '(passengers:0', 'crew:2)']
2 (passengers:0 crew:2)
['16', '(passengers:14', 'crew:2)']
16 (passengers:14 crew:2)
['24', '(passengers:18', 'cr

In [15]:
crash_df['Fatalities'] = crash_df.apply(lambda row: get_member_cnt(row['Fatalities']), axis=1)

['2', '(passengers:0', 'crew:2)']
2 (passengers:0 crew:2)
['8', '(passengers:6', 'crew:2)']
8 (passengers:6 crew:2)
['0', '(passengers:0', 'crew:0)']
0 (passengers:0 crew:0)
['13', '(passengers:?', 'crew:?)']
13 (passengers:? crew:?)
['24', '(passengers:22', 'crew:2)']
24 (passengers:22 crew:2)
['2', '(passengers:0', 'crew:2)']
2 (passengers:0 crew:2)
['49', '(passengers:44', 'crew:5)']
49 (passengers:44 crew:5)
['13', '(passengers:12', 'crew:1)']
13 (passengers:12 crew:1)
['5', '(passengers:0', 'crew:5)']
5 (passengers:0 crew:5)
['5', '(passengers:0', 'crew:5)']
5 (passengers:0 crew:5)
['9', '(passengers:6', 'crew:3)']
9 (passengers:6 crew:3)
['11', '(passengers:7', 'crew:4)']
11 (passengers:7 crew:4)
['17', '(passengers:15', 'crew:2)']
17 (passengers:15 crew:2)
['14', '(passengers:13', 'crew:1)']
14 (passengers:13 crew:1)
['2', '(passengers:0', 'crew:2)']
2 (passengers:0 crew:2)
['16', '(passengers:14', 'crew:2)']
16 (passengers:14 crew:2)
['24', '(passengers:18', 'crew:6)']
24 (pass

['4', '(passengers:0', 'crew:4)']
4 (passengers:0 crew:4)
['2', '(passengers:0', 'crew:2)']
2 (passengers:0 crew:2)
['16', '(passengers:?', 'crew:?)']
16 (passengers:? crew:?)
['2', '(passengers:1', 'crew:1)']
2 (passengers:1 crew:1)
['14', '(passengers:13', 'crew:1)']
14 (passengers:13 crew:1)
['49', '(passengers:44', 'crew:5)']
49 (passengers:44 crew:5)
['11', '(passengers:10', 'crew:1)']
11 (passengers:10 crew:1)
['8', '(passengers:8', 'crew:0)']
8 (passengers:8 crew:0)
['13', '(passengers:7', 'crew:6)']
13 (passengers:7 crew:6)
['5', '(passengers:3', 'crew:2)']
5 (passengers:3 crew:2)
['50', '(passengers:44', 'crew:6)']
50 (passengers:44 crew:6)
['2', '(passengers:2', 'crew:2)']
2 (passengers:2 crew:2)
['3', '(passengers:3', 'crew:0)']
3 (passengers:3 crew:0)
['33', '(passengers:27', 'crew:6)']
33 (passengers:27 crew:6)
['3', '(passengers:2', 'crew:1)']
3 (passengers:2 crew:1)
['4', '(passengers:3', 'crew:1)']
4 (passengers:3 crew:1)
['5', '(passengers:3', 'crew:2)']
5 (passengers:

In [16]:
crash_df_final=crash_df[["Date","Location","Operator","Aboard","Fatalities","Ground","Type","Summary"]].copy()

In [17]:
crash_df_final["Aboard"]= pd.to_numeric(crash_df_final['Aboard'],errors='coerce')
crash_df_final["Fatalities"]= pd.to_numeric(crash_df_final['Fatalities'],errors='coerce')

In [18]:
crash_df_final

,Date,Location,Operator,Aboard,Fatalities,Ground,Type,Summary
0,2009-01-01,"Lubock, Texas",Empire Airlines leased from FedEx,2,2,0,ATR-42-320,Crashed while attempting to land. The probable...
1,2009-01-04,Near Houma Louisiana,Petroleum Helicopters Inc,9,8,0,Sikorsky S-76C++,A helicopter bound for offshore oil fields wen...
2,2009-01-15,"New York, New York",US Airways,155,0,0,Airbus A320-214,The plane was taking off from La Guardia Airpo...
3,2009-01-15,"Adraskan district, Afghanistan",Afghan Army,13,13,0,Mil Mi-17,Either shot down or crashed due to bad weather.
4,2009-02-07,"Manacapuru, Brazil",Aerotaxi Manaus,28,24,0,Bandeirante EMB-110P1,The plane was being used as an air taxi to fer...
5,2009-02-07,"Trigoria, Italy",Air One Executive,2,2,0,Cessna 650 Citation III,"The plane, heading to Bologna to pick up a med..."
6,2009-02-12,"Clarence Center, New York",Continental Connection/Colgan Air,49,49,1,Bombardier DHC-8-402 Q400,The commuter plane crashed while attemptiong t...
7,2009-02-15,"Chanco, Chile",Flight Service,13,13,0,Bell UH-1H,The helicopter was carrying firemen to a nearb...
8,2009-02-15,"Near Isfahan/Shahin Shahr-Hesa Air Base, Iran",HESA,5,5,0,HESA IrAn-140-100,The aircraft crashed druing a training flight.
9,2009-02-20,"Luxor, Egypt",Aerolift,5,5,0,Antonov 12V,"While attemping to take off from Luxor, the ca..."


In [19]:
historical_df_final= historical_df.append(crash_df_final,ignore_index=True)

In [20]:
historical_df_final

,Date,Location,Operator,Aboard,Fatalities,Ground,Type,Summary
0,1908-09-17,"Fort Myer, Virginia",Military - U.S. Army,2.0,1.0,0,Wright Flyer III,"During a demonstration flight, a U.S. Army fly..."
1,1912-07-12,"AtlantiCity, New Jersey",Military - U.S. Navy,5.0,5.0,0,Dirigible,First U.S. dirigible Akron exploded just offsh...
2,1913-08-06,"Victoria, British Columbia, Canada",Private,1.0,1.0,0,Curtiss seaplane,The first fatal airplane accident in Canada oc...
3,1913-09-09,Over the North Sea,Military - German Navy,20.0,14.0,0,Zeppelin L-1 (airship),The airship flew into a thunderstorm and encou...
4,1913-10-17,"Near Johannisthal, Germany",Military - German Navy,30.0,30.0,0,Zeppelin L-2 (airship),Hydrogen gas which was being vented was sucked...
5,1915-03-05,"Tienen, Belgium",Military - German Navy,41.0,21.0,0,Zeppelin L-8 (airship),Crashed into trees while attempting to land af...
6,1915-09-03,"Off Cuxhaven, Germany",Military - German Navy,19.0,19.0,0,Zeppelin L-10 (airship),"Exploded and burned near Neuwerk Island, when..."
7,1916-07-28,"Near Jambol, Bulgeria",Military - German Army,20.0,20.0,0,Schutte-Lanz S-L-10 (airship),"Crashed near the Black Sea, cause unknown."
8,1916-09-24,"Billericay, England",Military - German Navy,22.0,22.0,0,Zeppelin L-32 (airship),Shot down by British aircraft crashing in flames.
9,1916-10-01,"Potters Bar, England",Military - German Navy,19.0,19.0,0,Zeppelin L-31 (airship),Shot down in flames by the British 39th Home D...


In [21]:
disk_engine = create_engine('sqlite:///aviation_accidents.sqlite')
crash_df_final.to_sql("Recent_data",disk_engine, if_exists='replace')

In [22]:
calc_final_summary=historical_df_final
calc_final_summary['Year']=historical_df_final['Date'].dt.year
calc_final_summary.to_sql('Historical_accidents', disk_engine, if_exists='replace')

In [23]:
historical_summary=calc_final_summary.groupby(["Year"])
historical_summary_report=pd.DataFrame(historical_summary["Year"].max())
historical_summary_report["Total_Accidents"]=historical_summary["Date"].count()
historical_summary_report["Total_Fatalities"]=historical_summary["Fatalities"].sum().astype(int)
historical_summary_report.reset_index(drop=True,inplace=True)
historical_summary_report.to_sql("Historical_summary",disk_engine,if_exists='replace')

In [24]:
# !pip install opencage

In [25]:
crash_df_final["Lat"] = ""
crash_df_final["Lng"] = ""
crash_df_final.head()

,Date,Location,Operator,Aboard,Fatalities,Ground,Type,Summary,Lat,Lng
0,2009-01-01,"Lubock, Texas",Empire Airlines leased from FedEx,2,2,0,ATR-42-320,Crashed while attempting to land. The probable...,,
1,2009-01-04,Near Houma Louisiana,Petroleum Helicopters Inc,9,8,0,Sikorsky S-76C++,A helicopter bound for offshore oil fields wen...,,
2,2009-01-15,"New York, New York",US Airways,155,0,0,Airbus A320-214,The plane was taking off from La Guardia Airpo...,,
3,2009-01-15,"Adraskan district, Afghanistan",Afghan Army,13,13,0,Mil Mi-17,Either shot down or crashed due to bad weather.,,
4,2009-02-07,"Manacapuru, Brazil",Aerotaxi Manaus,28,24,0,Bandeirante EMB-110P1,The plane was being used as an air taxi to fer...,,


In [26]:
test_crash = crash_df_final.head()

In [27]:
# Testing the for loop before running through all of the data
# Loop through the cities_pd and run a lat/long search for each city
for index, row in test_crash.iterrows():
    base_url = "https://api.opencagedata.com/geocode/v1/json?"
    
    Location = row['Location']
    # update address key value
    test_crash["Location"] = test_crash["Location"].str.replace(","," ")
    
#     try:
        # make request
    cities_lat_lng = requests.get(base_url, f"q={Location}&" +"api_key")

        # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
#     print(cities_lat_lng.url)

        # convert to json
    cities_lat_lng = cities_lat_lng.json()
#     print(cities_lat_lng)
        # If an error is experienced, skip the city
#         except:
#             print("City not found. Skipping...")
#             pass

    test_crash.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["lat"]
    test_crash.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["lng"]

# Visualize to confirm lat lng appear
test_crash

C:\Users\39319362\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\39319362\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Date,Location,Operator,Aboard,Fatalities,Ground,Type,Summary,Lat,Lng
0,2009-01-01,Lubock Texas,Empire Airlines leased from FedEx,2,2,0,ATR-42-320,Crashed while attempting to land. The probable...,31.2504,-99.2506
1,2009-01-04,Near Houma Louisiana,Petroleum Helicopters Inc,9,8,0,Sikorsky S-76C++,A helicopter bound for offshore oil fields wen...,29.678,-91.2871
2,2009-01-15,New York New York,US Airways,155,0,0,Airbus A320-214,The plane was taking off from La Guardia Airpo...,40.7309,-73.9872
3,2009-01-15,Adraskan district Afghanistan,Afghan Army,13,13,0,Mil Mi-17,Either shot down or crashed due to bad weather.,33.7054,61.9955
4,2009-02-07,Manacapuru Brazil,Aerotaxi Manaus,28,24,0,Bandeirante EMB-110P1,The plane was being used as an air taxi to fer...,-3.29968,-60.6214


In [28]:
# Loop through the cities_pd and run a lat/long search for each city
for index, row in crash_df_final.iterrows():
    base_url = "https://api.opencagedata.com/geocode/v1/json?"
    
    Location = row['Location']
    # update address key value
    crash_df_final["Location"] = crash_df_final["Location"].str.replace(","," ")
    
    try:
            # make request
        cities_lat_lng = requests.get(base_url, f"q={Location}&" +"api_key")

            # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    #     print(cities_lat_lng.url)

            # convert to json
        cities_lat_lng = cities_lat_lng.json()
    #     print(cities_lat_lng)

        crash_df_final.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["lat"]
        crash_df_final.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["lng"]
    
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...


In [29]:
# Visualize to confirm lat lng appear
crash_df_final

,Date,Location,Operator,Aboard,Fatalities,Ground,Type,Summary,Lat,Lng
0,2009-01-01,Lubock Texas,Empire Airlines leased from FedEx,2,2,0,ATR-42-320,Crashed while attempting to land. The probable...,31.2504,-99.2506
1,2009-01-04,Near Houma Louisiana,Petroleum Helicopters Inc,9,8,0,Sikorsky S-76C++,A helicopter bound for offshore oil fields wen...,29.678,-91.2871
2,2009-01-15,New York New York,US Airways,155,0,0,Airbus A320-214,The plane was taking off from La Guardia Airpo...,40.7309,-73.9872
3,2009-01-15,Adraskan district Afghanistan,Afghan Army,13,13,0,Mil Mi-17,Either shot down or crashed due to bad weather.,33.7054,61.9955
4,2009-02-07,Manacapuru Brazil,Aerotaxi Manaus,28,24,0,Bandeirante EMB-110P1,The plane was being used as an air taxi to fer...,-3.29968,-60.6214
5,2009-02-07,Trigoria Italy,Air One Executive,2,2,0,Cessna 650 Citation III,"The plane, heading to Bologna to pick up a med...",41.7647,12.4773
6,2009-02-12,Clarence Center New York,Continental Connection/Colgan Air,49,49,1,Bombardier DHC-8-402 Q400,The commuter plane crashed while attemptiong t...,43.0067,-78.6322
7,2009-02-15,Chanco Chile,Flight Service,13,13,0,Bell UH-1H,The helicopter was carrying firemen to a nearb...,-36.2951,-72.6801
8,2009-02-15,Near Isfahan/Shahin Shahr-Hesa Air Base Iran,HESA,5,5,0,HESA IrAn-140-100,The aircraft crashed druing a training flight.,,
9,2009-02-20,Luxor Egypt,Aerolift,5,5,0,Antonov 12V,"While attemping to take off from Luxor, the ca...",25.7021,32.6472


In [30]:
disk_engine = create_engine('sqlite:///aviation_accidents.sqlite')
crash_df_final.to_sql("Recent_data",disk_engine, if_exists='replace')

In [31]:
# The path to our CSV file
file = "AviationDataNTSB.csv"

# Read our Kickstarter data into pandas
df = pd.read_csv(file)
df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.Flight,Air.Carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Report.Status,Publication.Date
0,20181217X10943,Accident,GAA19CA098,2018-12-16,"Grangville, ID",United States,45.583611,-115.681667,PVT,NaN,...,Positioning,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Preliminary,2018-12-18
1,20181217X25746,Accident,GAA19CA097,2018-12-15,"MORIARTY, NM",United States,34.970000,-106.000000,NaN,MORIARTY MUNI,...,Personal,NaN,NaN,NaN,NaN,1.0,VMC,NaN,Preliminary,2018-12-20
2,20181213X41114,Accident,ERA19LA065,2018-12-13,"Punta Gorda, FL",United States,26.925278,-82.001111,PGD,Punta Gorda,...,Personal,NaN,NaN,NaN,1.0,NaN,VMC,APPROACH,Preliminary,2018-12-21
3,20181213X45528,Accident,CEN19FA044,2018-12-13,"Valparaiso, IN",United States,41.451667,-87.004444,VPZ,Porter County Rgnl,...,Personal,NaN,1.0,NaN,NaN,NaN,VMC,TAKEOFF,Preliminary,2018-12-19
4,20181214X90303,Accident,GAA19CA096,2018-12-11,"Hartford, WI",United States,43.330278,-88.326111,HXF,Hartford Muni,...,Personal,NaN,NaN,NaN,NaN,1.0,VMC,NaN,Preliminary,2018-12-18


In [32]:
df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.Damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.Flight', 'Air.Carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.Phase.of.Flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

In [33]:
reduced_aviation_df = df.loc[:, ["Event.Id","Event.Date","Location","Country","Latitude","Longitude","Aircraft.Damage","Aircraft.Category","Amateur.Built","Total.Fatal.Injuries","Total.Serious.Injuries","Total.Minor.Injuries","Total.Uninjured"]]

reduced_aviation_df

,Event.Id,Event.Date,Location,Country,Latitude,Longitude,Aircraft.Damage,Aircraft.Category,Amateur.Built,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
0,20181217X10943,2018-12-16,"Grangville, ID",United States,45.583611,-115.681667,Substantial,Airplane,No,NaN,NaN,NaN,1.0
1,20181217X25746,2018-12-15,"MORIARTY, NM",United States,34.970000,-106.000000,Substantial,Airplane,No,NaN,NaN,NaN,1.0
2,20181213X41114,2018-12-13,"Punta Gorda, FL",United States,26.925278,-82.001111,Substantial,Airplane,No,NaN,NaN,1.0,NaN
3,20181213X45528,2018-12-13,"Valparaiso, IN",United States,41.451667,-87.004444,Destroyed,Airplane,No,1.0,NaN,NaN,NaN
4,20181214X90303,2018-12-11,"Hartford, WI",United States,43.330278,-88.326111,Substantial,Airplane,No,NaN,NaN,NaN,1.0
5,20181210X21905,2018-12-10,"Chester, CT",United States,NaN,NaN,Substantial,Airplane,No,NaN,NaN,NaN,NaN
6,20181208X53536,2018-12-08,"Hesperia, CA",United States,34.376945,-117.316111,Substantial,Airplane,Yes,NaN,NaN,NaN,1.0
7,20181206X45049,2018-12-05,"Cornelia, GA",United States,NaN,NaN,Substantial,Airplane,No,NaN,NaN,NaN,2.0
8,20181204X03049,2018-12-04,"Reno, NV",United States,39.658889,-119.865833,Substantial,Airplane,No,NaN,NaN,NaN,2.0
9,20181212X01623,2018-12-04,"Moscow, PA",United States,41.326945,-75.591111,Substantial,Airplane,No,NaN,NaN,NaN,1.0


In [34]:
reduced_aviation_df = reduced_aviation_df.loc[(
    reduced_aviation_df["Amateur.Built"] == "No")]
reduced_aviation_df

,Event.Id,Event.Date,Location,Country,Latitude,Longitude,Aircraft.Damage,Aircraft.Category,Amateur.Built,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
0,20181217X10943,2018-12-16,"Grangville, ID",United States,45.583611,-115.681667,Substantial,Airplane,No,NaN,NaN,NaN,1.0
1,20181217X25746,2018-12-15,"MORIARTY, NM",United States,34.970000,-106.000000,Substantial,Airplane,No,NaN,NaN,NaN,1.0
2,20181213X41114,2018-12-13,"Punta Gorda, FL",United States,26.925278,-82.001111,Substantial,Airplane,No,NaN,NaN,1.0,NaN
3,20181213X45528,2018-12-13,"Valparaiso, IN",United States,41.451667,-87.004444,Destroyed,Airplane,No,1.0,NaN,NaN,NaN
4,20181214X90303,2018-12-11,"Hartford, WI",United States,43.330278,-88.326111,Substantial,Airplane,No,NaN,NaN,NaN,1.0
5,20181210X21905,2018-12-10,"Chester, CT",United States,NaN,NaN,Substantial,Airplane,No,NaN,NaN,NaN,NaN
7,20181206X45049,2018-12-05,"Cornelia, GA",United States,NaN,NaN,Substantial,Airplane,No,NaN,NaN,NaN,2.0
8,20181204X03049,2018-12-04,"Reno, NV",United States,39.658889,-119.865833,Substantial,Airplane,No,NaN,NaN,NaN,2.0
9,20181212X01623,2018-12-04,"Moscow, PA",United States,41.326945,-75.591111,Substantial,Airplane,No,NaN,NaN,NaN,1.0
10,20181204X93009,2018-12-03,"Monroe, NC",United States,35.017500,-80.621945,Substantial,Airplane,No,NaN,NaN,NaN,1.0


In [35]:
reduced_aviation_df = reduced_aviation_df.loc[(
    reduced_aviation_df["Aircraft.Category"] == "Airplane")]
reduced_aviation_df

,Event.Id,Event.Date,Location,Country,Latitude,Longitude,Aircraft.Damage,Aircraft.Category,Amateur.Built,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
0,20181217X10943,2018-12-16,"Grangville, ID",United States,45.583611,-115.681667,Substantial,Airplane,No,NaN,NaN,NaN,1.0
1,20181217X25746,2018-12-15,"MORIARTY, NM",United States,34.970000,-106.000000,Substantial,Airplane,No,NaN,NaN,NaN,1.0
2,20181213X41114,2018-12-13,"Punta Gorda, FL",United States,26.925278,-82.001111,Substantial,Airplane,No,NaN,NaN,1.0,NaN
3,20181213X45528,2018-12-13,"Valparaiso, IN",United States,41.451667,-87.004444,Destroyed,Airplane,No,1.0,NaN,NaN,NaN
4,20181214X90303,2018-12-11,"Hartford, WI",United States,43.330278,-88.326111,Substantial,Airplane,No,NaN,NaN,NaN,1.0
5,20181210X21905,2018-12-10,"Chester, CT",United States,NaN,NaN,Substantial,Airplane,No,NaN,NaN,NaN,NaN
7,20181206X45049,2018-12-05,"Cornelia, GA",United States,NaN,NaN,Substantial,Airplane,No,NaN,NaN,NaN,2.0
8,20181204X03049,2018-12-04,"Reno, NV",United States,39.658889,-119.865833,Substantial,Airplane,No,NaN,NaN,NaN,2.0
9,20181212X01623,2018-12-04,"Moscow, PA",United States,41.326945,-75.591111,Substantial,Airplane,No,NaN,NaN,NaN,1.0
10,20181204X93009,2018-12-03,"Monroe, NC",United States,35.017500,-80.621945,Substantial,Airplane,No,NaN,NaN,NaN,1.0


In [36]:
aviation_final_df = reduced_aviation_df.loc[(
    reduced_aviation_df["Country"] == "United States")]
aviation_final_df

,Event.Id,Event.Date,Location,Country,Latitude,Longitude,Aircraft.Damage,Aircraft.Category,Amateur.Built,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
0,20181217X10943,2018-12-16,"Grangville, ID",United States,45.583611,-115.681667,Substantial,Airplane,No,NaN,NaN,NaN,1.0
1,20181217X25746,2018-12-15,"MORIARTY, NM",United States,34.970000,-106.000000,Substantial,Airplane,No,NaN,NaN,NaN,1.0
2,20181213X41114,2018-12-13,"Punta Gorda, FL",United States,26.925278,-82.001111,Substantial,Airplane,No,NaN,NaN,1.0,NaN
3,20181213X45528,2018-12-13,"Valparaiso, IN",United States,41.451667,-87.004444,Destroyed,Airplane,No,1.0,NaN,NaN,NaN
4,20181214X90303,2018-12-11,"Hartford, WI",United States,43.330278,-88.326111,Substantial,Airplane,No,NaN,NaN,NaN,1.0
5,20181210X21905,2018-12-10,"Chester, CT",United States,NaN,NaN,Substantial,Airplane,No,NaN,NaN,NaN,NaN
7,20181206X45049,2018-12-05,"Cornelia, GA",United States,NaN,NaN,Substantial,Airplane,No,NaN,NaN,NaN,2.0
8,20181204X03049,2018-12-04,"Reno, NV",United States,39.658889,-119.865833,Substantial,Airplane,No,NaN,NaN,NaN,2.0
9,20181212X01623,2018-12-04,"Moscow, PA",United States,41.326945,-75.591111,Substantial,Airplane,No,NaN,NaN,NaN,1.0
10,20181204X93009,2018-12-03,"Monroe, NC",United States,35.017500,-80.621945,Substantial,Airplane,No,NaN,NaN,NaN,1.0


In [37]:
aviation_final = aviation_final_df.fillna(value="Unknown")
aviation_final

,Event.Id,Event.Date,Location,Country,Latitude,Longitude,Aircraft.Damage,Aircraft.Category,Amateur.Built,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
0,20181217X10943,2018-12-16,"Grangville, ID",United States,45.5836,-115.682,Substantial,Airplane,No,Unknown,Unknown,Unknown,1
1,20181217X25746,2018-12-15,"MORIARTY, NM",United States,34.97,-106,Substantial,Airplane,No,Unknown,Unknown,Unknown,1
2,20181213X41114,2018-12-13,"Punta Gorda, FL",United States,26.9253,-82.0011,Substantial,Airplane,No,Unknown,Unknown,1,Unknown
3,20181213X45528,2018-12-13,"Valparaiso, IN",United States,41.4517,-87.0044,Destroyed,Airplane,No,1,Unknown,Unknown,Unknown
4,20181214X90303,2018-12-11,"Hartford, WI",United States,43.3303,-88.3261,Substantial,Airplane,No,Unknown,Unknown,Unknown,1
5,20181210X21905,2018-12-10,"Chester, CT",United States,Unknown,Unknown,Substantial,Airplane,No,Unknown,Unknown,Unknown,Unknown
7,20181206X45049,2018-12-05,"Cornelia, GA",United States,Unknown,Unknown,Substantial,Airplane,No,Unknown,Unknown,Unknown,2
8,20181204X03049,2018-12-04,"Reno, NV",United States,39.6589,-119.866,Substantial,Airplane,No,Unknown,Unknown,Unknown,2
9,20181212X01623,2018-12-04,"Moscow, PA",United States,41.3269,-75.5911,Substantial,Airplane,No,Unknown,Unknown,Unknown,1
10,20181204X93009,2018-12-03,"Monroe, NC",United States,35.0175,-80.6219,Substantial,Airplane,No,Unknown,Unknown,Unknown,1


In [38]:
disk_engine = create_engine('sqlite:///aviation_accidents.sqlite')
aviation_final.to_sql('NTSB_accidents', disk_engine, if_exists='replace')

In [39]:
def investigate(s):
   
    data1= {
        'Pilot Error' : ['IMPROPER PROCEDURE','CONTROLLED FLIGHT INTO TERRAIN','DESCENDING BELOW MINIMA',
                         'SPATIAL DISORIENTATION','PREMATURE DESCENT','EXCESSIVE LANDING SPEED','MISSED RUNWAY',
                         'FUEL STARVATION','NAVIGATION ERROR','TAKEOFF','LANDING','AIR COLLISION'],
        
        'Mechanical' :['ENGINE FAILURE','EQUIPMENT FAILURE','STRUCTURAL FAILURE','DESIGN FLAW','MAINTENANCE ERROR'],
        
        'Weather' :['SEVERE TURBULENCE','WINDSHEAR','MOUNTAIN WAVE','POOR VISIBILITY','SEVERE WINDS','THUNDERSTORMS',
                   'LIGHTNING STRIKE','ICING'],
        'Sabotage' : ['HIJACKING','SHOT DOWN','EXPLOSIVE DEVISE ABOARD'],
        
        'Other' :[ 'ATC ERROR','GROUND CREW ERROR','OVERLOADED','IMPROPERLY LOADED CARGO','BIRD STRIKE','FUEL CONTAMINATION',
                   'PILOT INCAPACITATION','OBSTRUCTION ON RUNWAY','SMOKE','FIRE']        
    }

    res = []
    for el, words in data1.items():
        res += [el for word in words if word in s]
        
    return list(set(res))

mp_df = historical_df_final.copy()
mp_df.Operator=mp_df.Operator.str.upper()


mp_df['Military'] = mp_df.Operator.str.contains('MILITARY')|mp_df.Operator.str.contains('AIR FORCE')
mp_df['Passenger'] = mp_df.Military == False

mp_df_passenger=mp_df[mp_df["Passenger"].isin([True])]


In [40]:
mp_df_passenger

,Date,Location,Operator,Aboard,Fatalities,Ground,Type,Summary,Year,Military,Passenger
2,1913-08-06,"Victoria, British Columbia, Canada",PRIVATE,1.0,1.0,0,Curtiss seaplane,The first fatal airplane accident in Canada oc...,1913,False,True
21,1918-12-16,"Elizabeth, New Jersey",US AERIAL MAIL SERVICE,1.0,1.0,0,De Havilland DH-4,NaN,1918,False,True
22,1919-05-25,"Cleveland, Ohio",US AERIAL MAIL SERVICE,1.0,1.0,0,De Havilland DH-4,Caught fire in midair. The pilot leaped from t...,1919,False,True
23,1919-07-19,"Dix Run, Pennsylvania",US AERIAL MAIL SERVICE,1.0,1.0,0,De Havilland DH-4,NaN,1919,False,True
24,1919-10-02,"Newcastle, England",AIRCRAFT TRANSPORT AND TRAVEL,1.0,1.0,0,De Havilland DH-4,NaN,1919,False,True
25,1919-10-14,"Cantonsville, Maryland",US AERIAL MAIL SERVICE,1.0,1.0,0,Curtiss R-4LM,NaN,1919,False,True
26,1919-10-20,English Channel,AIRCRAFT TRANSPORT AND TRAVEL,NaN,NaN,NaN,De Havilland DH-4,NaN,1919,False,True
27,1919-10-30,"Long Valley, New Jersey",US AERIAL MAIL SERVICE,1.0,1.0,0,De Havilland DH.4,NaN,1919,False,True
28,1920-03-10,"New Paris, Indiana",US AERIAL MAIL SERVICE,1.0,1.0,0,De Havilland DH-4,NaN,1920,False,True
29,1920-03-30,"Newark, New Jersey",US AERIAL MAIL SERVICE,1.0,1.0,0,De Havilland DH-4,NaN,1920,False,True


In [41]:
mp_df = historical_df_final.copy()
mp_df.Operator=mp_df.Operator.str.upper()


mp_df['Military'] = mp_df.Operator.str.contains('MILITARY')|mp_df.Operator.str.contains('AIR FORCE')
mp_df['Passenger'] = mp_df.Military == False

mp_df_passenger=mp_df[mp_df["Passenger"].isin([True])]

mp_df_passenger=mp_df_passenger[mp_df_passenger["Year"].isin (["1960","1961","1962","1963","1964","1965","1966","1967","1968","1969","1970","1971",
                                           "1972","1973","1974" "1975","1976","1977","1978",\
                                           "1979","1980","1981","1982","1983","1984","1985","1986","1987","1988","1989",\
                                           "1990","1991","1992","1993","1994","1995","1996","1997","1998","1999",\
                                           "2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010",\
                                                         "2011","2012","2013","2014","2015","2016","2017","2018","2019"])]

In [42]:
mp_df_passenger

,Date,Location,Operator,Aboard,Fatalities,Ground,Type,Summary,Year,Military,Passenger
1237,1970-02-13,"Pin-la-Garenne, France",AVIA-TAXI,10.0,10.0,0,Cessna 402,NaN,1970,False,True
1577,1962-01-13,"Near Tippi, Ethiopia",ETHIOPIAN AIRLINES,18.0,5.0,1,Douglas DC-3,Swerved off the runway while taking off and st...,1962,False,True
1638,1960-01-03,"Taksing, India",INDIAN AIRLINES,9.0,9.0,0,Douglas C-47A,The cargo plane crashed while dropping supplie...,1960,False,True
1639,1960-01-06,"Wilmington/Bolivia, North Carolina",NATIONAL AIRLINES,34.0,34.0,0,Douglas DC-6B,"The plane disintegrated in flight at 18,000 fe...",1960,False,True
1641,1960-01-18,"Charles City, Virginia",CAPITAL AIRLINES,50.0,50.0,0,Vickers Viscount 745D,The aircraft crashed and burned in a wooded ar...,1960,False,True
1642,1960-01-19,"Ankara, Turkey",SCANDINAVIAN AIRLINES (SAS),42.0,42.0,0,Sud Aviation Caravelle 1,"While on a flight from Istanbul to Ankara, Tur...",1960,False,True
1643,1973-05-11,"Near Semipalatinsk, USSR",AEROFLOT,61.0,61.0,0,Ilyushin IL-18B,NaN,1973,False,True
1644,1960-01-21,"Montego Bay, Jamacia",AVIANCA,46.0,37.0,0,Lockheed 1049E Super Constellation,A heavy landing led to structural failure and ...,1960,False,True
1645,1960-01-26,"Off Bathurst Island, Australia",TRANSPORTES AÉREOS DE TIMOR,9.0,9.0,0,de Havilland DH-114 Heron 2D,Radio contact was lost with the aircraft befor...,1960,False,True
1646,1960-02-05,"Laguna de Huana-Costa , Boliva",LLOYD AEREO BOLIVIANO,59.0,59.0,0,Douglas DC-4,Crashed 15 minutes after taking off. Engine fire.,1960,False,True


In [43]:
mp_df_passenger['Summary'].fillna('', inplace=True)
mp_df_passenger['Summary'].replace('practicing','practice',inplace=True)
all_values = []
for s in mp_df_passenger['Summary']:
    all_values += investigate(s.upper())
    
print(all_values)

['Weather', 'Mechanical', 'Other', 'Pilot Error', 'Other', 'Pilot Error', 'Mechanical', 'Mechanical', 'Pilot Error', 'Mechanical', 'Pilot Error', 'Pilot Error', 'Mechanical', 'Other', 'Other', 'Mechanical', 'Mechanical', 'Weather', 'Pilot Error', 'Other', 'Pilot Error', 'Pilot Error', 'Pilot Error', 'Weather', 'Pilot Error', 'Pilot Error', 'Pilot Error', 'Pilot Error', 'Pilot Error', 'Pilot Error', 'Pilot Error', 'Other', 'Pilot Error', 'Pilot Error', 'Pilot Error', 'Other', 'Mechanical', 'Pilot Error', 'Pilot Error', 'Mechanical', 'Pilot Error', 'Pilot Error', 'Other', 'Pilot Error', 'Pilot Error', 'Other', 'Pilot Error', 'Pilot Error', 'Pilot Error', 'Pilot Error', 'Other', 'Pilot Error', 'Pilot Error', 'Mechanical', 'Other', 'Pilot Error', 'Weather', 'Pilot Error', 'Pilot Error', 'Pilot Error', 'Pilot Error', 'Pilot Error', 'Pilot Error', 'Weather', 'Weather', 'Weather', 'Pilot Error', 'Weather', 'Pilot Error', 'Weather', 'Pilot Error', 'Other', 'Mechanical', 'Pilot Error', 'Pilot E

In [44]:
#No_summ_df=historical_df_final[historical_df_final["Summary"] == ""]
from collections import Counter
count_causes = Counter(all_values)
output_list_final= []
print(count_causes)

Counter({'Pilot Error': 850, 'Other': 311, 'Weather': 212, 'Mechanical': 128, 'Sabotage': 42})


In [45]:
for i in count_causes:
    output_list={}
    output_list["Cause"]=i
    output_list["Count"]=count_causes[i]
    output_list_final.append(output_list)
causes_df=pd.DataFrame(output_list_final)

In [46]:
disk_engine = create_engine('sqlite:///aviation_accidents.sqlite')

causes_df.to_sql("Count_by_causes",disk_engine, if_exists='replace')

In [47]:
causes_df["Percentage"]=(causes_df["Count"]/causes_df["Count"].sum())*100

In [48]:
causes_df

,Cause,Count,Percentage
0,Weather,212,13.739469
1,Mechanical,128,8.295528
2,Other,311,20.155541
3,Pilot Error,850,55.087492
4,Sabotage,42,2.721970


In [49]:
data_file=("World Departures.csv")
departure_df=pd.read_csv(data_file)
departure_df=departure_df[["1970","1971","1973","1974","1975","1976","1977","1978",\
                        "1979","1980","1981","1982","1983","1984","1985","1986","1987","1988","1989",\
                        "1990","1991","1992","1993","1994","1995","1996","1997","1998","1999",\
                        "2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010",\
                        "2011","2012","2013","2014","2015","2016","2017"]]
departure_df

,1970,1971,1973,1974,1975,1976,1977,1978,1979,1980,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,9448300,9504500,9764900,9254900,9244700,9562900,9915900,10183700,10653400,10704900,...,25498093,26122523,29637929.54,30564978.1,30771384.59,31464398.88,32339781.98,33362839,34687965,35842803


In [50]:
departure_df = pd.melt(departure_df, var_name='Year', value_name='Value')

disk_engine = create_engine('sqlite:///aviation_accidents.sqlite')
departure_df.to_sql("Departure_data",disk_engine, if_exists='replace')

In [51]:
departure_df

,Year,Value
0,1970,9448300.00
1,1971,9504500.00
2,1973,9764900.00
3,1974,9254900.00
4,1975,9244700.00
5,1976,9562900.00
6,1977,9915900.00
7,1978,10183700.00
8,1979,10653400.00
9,1980,10704900.00
